In [22]:
import pickle
repo_id = '45526e5b-f544-4016-8381-f88f5ca095ea'
G = pickle.load(open(repo_id+'/state_0.pkl', 'rb'))
print(G)

DiGraph with 16207 nodes and 36362 edges


In [23]:
G.nodes(data=True)['Superagent']

KeyError: 'Superagent'

In [24]:
# for nodes with type method
for n in G.nodes(data=True):
    try:
        if n[1]['type'] == 'method':
            # get the predecessors
            preds = G.predecessors(n[0])
            print(n[0])
            [print(G.nodes[p]['name']) for p in preds]
            break
    except:
        pass

__init__
SimpleMongoReader
RocksetVectorStore
SQLJoinQueryEngine
BaseStructStoreIndex
SentenceSplitter
TreeSummarize
IPYNBReader
AzureOpenAIEmbedding
Perplexity
CassandraVectorStore
LlamaCPP
FaissVectorStore
LangchainPromptTemplate
LLMMultiSelector
BagelVectorStore
AimCallback
DFFullProgram
OpenAIQuestionGenerator
CondenseQuestionChatEngine
DocumentSummaryIndex
GoogleVectorStore
FaissReader
MockChatLLM
OpenLLM
ZepVectorStore
LinearLayer
Accumulate
EvaporateExtractor
CondensePlusContextChatEngine
PaLM
PGVectoRsStore
NLStructStoreQueryEngine
ElasticsearchReader
SlackReader
OpenAIEmbedding
ReActAgent
MongoDocumentStore
OpenAI
Neo4jGraphStore
Anthropic
MetaProvider
MultiModalVectorIndexRetriever
Refine
AwadbReader
GradientBaseModelLLM
GradientFinetuneEngine
VectorStoreIndex
OpenAIFinetuneEngine
LangchainEmbedding
SQLDatabase
FirestoreKVStore
SummaryExtractor
LLMPredictor
OnDemandLoaderTool
WatsonX
PandasIndex
MetalVectorStore
PGVectorSQLParser
DocArrayInMemoryVectorStore
DiscordReader
Toke

In [25]:
import os
from chromadb import PersistentClient, HttpClient
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
load_dotenv()

def return_collection(path=None, collection_name=None):
    assert path is not None, "Path isn't specified"
    assert collection_name is not None, "Collection name isn't specified"
    chroma_client = PersistentClient(path=path)
    emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.getenv('OPENAI_API_KEY'),
        model_name="text-embedding-ada-002"
    )
    collection = chroma_client.get_or_create_collection(name=collection_name, embedding_function=emb_fn, metadata={"hnsw:space": "cosine"})
    return collection

path_ = f"{repo_id}/meta/storage"
explanations = return_collection(path=path_, collection_name="explanations")
explanations.count()

2097

In [15]:
code = return_collection(path=path_, collection_name="code")
code.count()

25

In [16]:
triplets = return_collection(path=path_, collection_name="triplets")
triplets.count()

186

In [42]:
query = "What"
res = triplets.query(
    query_texts=[query],
    n_results=10
)
[r for r in res['documents']]

[['template_upload_pipeline function-call chunking_pipeline',
  'template_upload_pipeline function-call embed_pipeline',
  'template_upload_pipeline EXTERNAL_DEPENDENCY chunking_pipeline_0',
  'template_upload_pipeline EXTERNAL_DEPENDENCY embed_pipeline_0',
  'update_database.py function template_upload_pipeline',
  'template_upload_pipeline function-call cot_pipeline',
  'template_upload_pipeline EXTERNAL_DEPENDENCY cot_pipeline_0',
  'chunking_pipeline EXTERNAL_DEPENDENCY open_files_0',
  'embed.py function embed_pipeline',
  'chunking_pipeline EXTERNAL_DEPENDENCY split_files_1']]

In [46]:
query = "What is happening with the template upload pipeline?"
res = code.query(
    query_texts=[query],
    n_results=10
)
[r for r in res['metadatas']]

[[{'containedin': 'update_database.py',
   'index': '0',
   'name': 'template_upload_pipeline',
   'type': 'function'},
  {'containedin': 'dfs.py',
   'index': '2',
   'name': 'cot_pipeline',
   'type': 'function'},
  {'containedin': 'embed.py',
   'index': '3',
   'name': 'embed_pipeline',
   'type': 'function'},
  {'containedin': 'chunker.py',
   'index': '5',
   'name': 'chunking_pipeline',
   'type': 'function'},
  {'containedin': 'embed.py',
   'index': '0',
   'name': 'get_embedding',
   'type': 'function'},
  {'containedin': 'agents.py',
   'index': '1',
   'name': 'create_assistant',
   'type': 'function'},
  {'containedin': 'embed.py',
   'index': '1',
   'name': 'open_data_json',
   'type': 'function'},
  {'containedin': 'agents.py',
   'index': '2',
   'name': 'create_thread',
   'type': 'function'},
  {'containedin': 'search.py',
   'index': '1',
   'name': 'open_embeddings_json',
   'type': 'function'},
  {'containedin': 'search.py',
   'index': '0',
   'name': 'open_data_

In [29]:
query = "class to load pdf files into a store?"
res = explanations.query(
    query_texts=[query],
    n_results=10
)
[r for r in (res['metadatas'], res['documents'])]

[[[{'name': 'PDFReader', 'type': 'class'},
   {'name': 'docstore', 'type': 'folder'},
   {'name': 'BaseDocumentStore', 'type': 'class'},
   {'name': 'BaseReader', 'type': 'class'},
   {'name': 'DocArrayVectorStore', 'type': 'class'},
   {'name': 'from_uri', 'type': 'method'},
   {'name': 'DocxReader', 'type': 'class'},
   {'name': 'VectorStoreQueryOutputParser', 'type': 'class'},
   {'name': 'MilvusVectorStore', 'type': 'class'},
   {'name': 'DocumentStoreType', 'type': 'class'}]],
 [['The code snippet defines a class called PDFReader that is a subclass of BaseReader. The class is used for parsing PDF files. It has two methods: __init__ and load_data. The __init__ method is used for initializing the PDFReader object, and the load_data method is used for loading data from the PDF file.',
   'The code defines a method called "docstore" that belongs to a class. This method returns the value of a variable called "_docstore", which is expected to be an instance of the "BaseDocumentStore" cl

In [49]:
for edge in G.edges(data=True):
    if 'template_upload_pipeline' in edge:
        print(edge)

('update_database.py', 'template_upload_pipeline', {'type': 'function'})
('template_upload_pipeline', 'chunking_pipeline', {'type': 'function-call'})
('template_upload_pipeline', 'embed_pipeline', {'type': 'function-call'})
('template_upload_pipeline', 'cot_pipeline', {'type': 'function-call'})
('template_upload_pipeline', 'chunking_pipeline_0', {'type': 'EXTERNAL_DEPENDENCY'})
('template_upload_pipeline', 'embed_pipeline_0', {'type': 'EXTERNAL_DEPENDENCY'})
('template_upload_pipeline', 'cot_pipeline_0', {'type': 'EXTERNAL_DEPENDENCY'})


In [33]:
# store the output connections, what comes out and where it coes in (a train of events). How the functions are being called.

In [50]:
for edge in G.edges(data=True):
    if 'chunking_pipeline' in edge:
        print(edge)

('chunker.py', 'chunking_pipeline', {'type': 'function'})
('chunking_pipeline', 'open_files', {'type': 'function-call'})
('chunking_pipeline', 'extract_education_and_experience', {'type': 'function-call'})
('chunking_pipeline', 'store_data', {'type': 'function-call'})
('chunking_pipeline', 'open_files_0', {'type': 'EXTERNAL_DEPENDENCY'})
('chunking_pipeline', 'split_files_1', {'type': 'EXTERNAL_DEPENDENCY'})
('chunking_pipeline', 'extract_education_and_experience_0', {'type': 'EXTERNAL_DEPENDENCY'})
('chunking_pipeline', 'store_data_0', {'type': 'EXTERNAL_DEPENDENCY'})
('template_upload_pipeline', 'chunking_pipeline', {'type': 'function-call'})
